In [3]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
train_data = pd.read_csv('/kaggle/input/cleaned-datasets/cleaned_train.csv')  # Replace with the path to your training data
test_data = pd.read_csv('/kaggle/input/cleaned-datasets/cleaned_test.csv')  # Replace with the path to your test data

train_data.head()

,Month,Age,Profession,Income_Annual,Base_Salary_PerMonth,Total_Bank_Accounts,Total_Credit_Cards,Rate_Of_Interest,Total_Current_Loans,Delay_from_due_date,...,Current_Debt_Outstanding,Ratio_Credit_Utilization,Credit_History_Age,Payment_of_Min_Amount,Per_Month_EMI,Monthly_Investment,Payment_Behaviour,Monthly_Balance,Credit_Score,Loan_Count
0,7,51,11,101583.48,8648.290000,5,7,10,4,8,...,50.93,34.462154,289,0,190.811017,630.015789,3,314.002193,1,4
1,1,23,14,101926.95,8635.912500,4,4,9,1,13,...,1058.00,39.693812,245,0,70.587681,662.803927,4,410.199642,1,1
2,2,49,14,158871.12,12962.260000,0,4,8,1,8,...,576.48,39.367225,228,0,86.905860,746.805985,4,742.514154,1,1
3,6,40,3,60379.28,4804.606667,5,6,18,3,15,...,725.39,29.061701,205,0,90.906385,166.418658,1,473.135623,1,3
4,5,17,0,50050.83,4085.902500,9,10,20,5,28,...,3419.10,30.386321,54,1,190.445060,56.789441,0,401.355749,0,5


In [4]:
test_data.head()

,Month,Age,Profession,Income_Annual,Base_Salary_PerMonth,Total_Bank_Accounts,Total_Credit_Cards,Rate_Of_Interest,Total_Current_Loans,Delay_from_due_date,...,Current_Debt_Outstanding,Ratio_Credit_Utilization,Credit_History_Age,Payment_of_Min_Amount,Per_Month_EMI,Monthly_Investment,Payment_Behaviour,Monthly_Balance,Loan_Count,ID
0,0,40,9,44386.470,3703.872500,10,6,26,9,34,...,1275.46,25.537989,110,1,298.848530,50.620287,1,270.918433,9,0x2145
1,1,34,2,19506.730,1491.560833,9,7,22,6,47,...,1594.83,24.274661,178,1,59.134042,149.687764,5,230.334277,6,0x7d59
2,5,39,5,39962.380,3151.198333,6,7,22,5,18,...,1444.91,38.856838,0,1,52463.000000,147.403727,4,357.954330,5,0xe753
3,2,28,9,7477.175,713.097917,9,5,15,8,55,...,4944.58,36.125353,88,1,35.180443,62.795114,5,288.762718,8,0x19813
4,3,44,1,41771.020,3293.918333,6,10,17,7,35,...,3524.73,24.735922,98,1,127.617609,109.442464,1,342.331760,7,0x1dc5e


In [7]:
# Define the feature columns and target column
X_train = train_data.drop(columns=['Credit_Score'])  
y_train = train_data['Credit_Score']  

# Handle missing values in the training data (mean imputation)
X_train = X_train.fillna(X_train.mean())

# Preprocess the test data (no 'Credit_Score' column, and no ID column)
X_test = test_data.drop(columns=['ID']) 
X_test = X_test.fillna(X_test.mean())  

# Initialize and train the SVM classifier
svm_model = SVC(kernel='rbf')  
svm_model.fit(X_train, y_train)

# Make predictions using the trained SVM model
predictions = svm_model.predict(X_test)

# Add the predictions to the test data dataframe
test_data['Predicted_Credit_Score'] = predictions

# Output the predictions along with the 'ID' column for identification
print(test_data[['ID', 'Predicted_Credit_Score']])

# Optional: Calculate and print the accuracy if true labels are available in the test data
if 'Credit_Score' in test_data.columns:
    y_test = test_data['Credit_Score']  
    accuracy = accuracy_score(y_test, predictions)
    print(f"Accuracy of the SVM model on test data: {accuracy:.4f}")
else:
    print("True labels not available in test data. Accuracy cannot be calculated.")

            ID  Predicted_Credit_Score
0       0x2145                       1
1       0x7d59                       1
2       0xe753                       1
3      0x19813                       0
4      0x1dc5e                       0
...        ...                     ...
19995   0x74ee                       1
19996   0xd036                       1
19997   0xba16                       1
19998   0xce25                       1
19999   0x2324                       1

[20000 rows x 2 columns]
True labels not available in test data. Accuracy cannot be calculated.


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns

# Define the feature columns and target column
X = train_data.drop(columns=['Credit_Score'])  # Features
y = train_data['Credit_Score']  # Target

# Handle missing values in the training data (mean imputation)
X = X.fillna(X.mean())

# Split the data into training and validation sets (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Initialize SVM with specified hyperparameters
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=1)

# Train the SVM model
svm_model.fit(X_train, y_train)

# Evaluate on the training data
train_predictions = svm_model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)
print(f"Training Accuracy: {train_accuracy:.4f}")

# Evaluate on the validation data
val_predictions = svm_model.predict(X_val)
val_accuracy = accuracy_score(y_val, val_predictions)
print(f"Validation Accuracy: {val_accuracy:.4f}")

Training Accuracy: 0.6895
Validation Accuracy: 0.6760


"\n# Retrain the model on the full training data\nX_full_scaled = scaler.fit_transform(X)  # Standardize the full data\nsvm_model.fit(X_full_scaled, y)  # Retrain\n\n# Preprocess the test data (no 'Credit_Score' column, and no 'ID' column)\nX_test = test_data.drop(columns=['ID'])\nX_test = X_test.fillna(X_test.mean())\nX_test_scaled = scaler.transform(X_test)  # Standardize test data\n\n# Make predictions on the test data\npredictions = svm_model.predict(X_test_scaled)\n\n# Add the predictions to the test data dataframe\ntest_data['Predicted_Credit_Score'] = predictions\n\n# Output predictions along with 'ID' column\nprint(test_data[['ID', 'Predicted_Credit_Score']])\n"